### Yolo11 tests

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os , sys
import json
sys.path.append('..')
from pathlib import Path
cur_path = Path(os.getcwd()).parent
sam2_path = cur_path / 'sam2_octron'
sys.path.append(cur_path.as_posix())
from matplotlib import pyplot as plt
import cmasher as cmr
import numpy as np
import seaborn as sns
sns.set_theme(style='white')
%config InlineBackend.figure_format = 'retina'

In [3]:
# Object organizer
from octron.sam2_octron.object_organizer import load_object_organizer
# #### Importing additional stuff 
# from skimage import measure
# from skimage.draw import polygon2mask

In [13]:
from napari_pyav._reader import FastVideoReader


In [4]:
from ultralytics import YOLO
from ultralytics import settings
settings.update({'sync': False,'hub':False, })

In [5]:
settings.update({'datasets_dir': '','weights_dir':'', 'runs_dir':'OCTRON RUNS'})

In [6]:
# Presaved model (YOLOv11)
path_to_model = Path('/Users/horst/Documents/python/OCTRON/octron/yolo_octron/yolo11l-seg.pt')
# Output data from annotation in OCTRON
path_to_json_organizer = Path('/Users/horst/Downloads/octron_project/object_organizer.json')

In [8]:
organization = load_object_organizer(path_to_json_organizer)  
assert organization is not None
print(organization['entries'].keys())

📖 Octron object organizer loaded from /Users/horst/Downloads/octron_project/object_organizer.json
dict_keys(['0', '1', '2'])


In [12]:
for entry in organization['entries'].values():
    print(entry['label'])
    print(entry['label_id'])
    print(entry['prediction_layer_metadata'])

wormsy
0
{'name': 'wormsy masks', 'type': 'Labels', 'data_shape': [2036, 1000, 1000], 'ndim': 3, 'visible': True, 'opacity': 0.4, 'colormap_name': 'custom', 'zarr_path': '/Users/horst/Downloads/octron_project/wormsy masks.zip'}
handle
1
{'name': 'handle 1 masks', 'type': 'Labels', 'data_shape': [2036, 1000, 1000], 'ndim': 3, 'visible': True, 'opacity': 0.4, 'colormap_name': 'custom', 'zarr_path': '/Users/horst/Downloads/octron_project/handle 1 masks.zip'}
handle
1
{'name': 'handle 2 masks', 'type': 'Labels', 'data_shape': [2036, 1000, 1000], 'ndim': 3, 'visible': True, 'opacity': 0.4, 'colormap_name': 'custom', 'zarr_path': '/Users/horst/Downloads/octron_project/handle 2 masks.zip'}


In [16]:
video_path = Path('/Users/horst/Downloads/octron_project/first_half.mp4')
video_data = FastVideoReader(video_path, read_format='rgb24')

In [19]:
video_data.shape

(2036, 1000, 1000, 3)

In [23]:
#model = YOLO(path_to_model)  # load a pretrained model (recommended for training)


In [ ]:
# # Train/val/test sets as 
# 1) dir: path/to/imgs, 
# 2) file: path/to/imgs.txt, or list: [path/to/imgs1, path/to/imgs2, ..]
# path: ../datasets/coco8-seg # dataset root dir (absolute or relative; if relative, it's relative to default datasets_dir)
# train: images/train # train images (relative to 'path') 4 images
# val: images/val # val images (relative to 'path') 4 images
# test: # test images (optional)
